Please see the attachment for a hand-drawn plot of the first three levels of the tree.

Training error: 0.0
Test Error: 0.171

training error(before pruning): 0.0

test error(before pruning): 0.171

validation error (before pruning): 0.178.



training error(after 1 pruning): 0.0205

validation error (after 1 pruning): 0.163.

test error(after 1 pruning): 0.157



training error(after 2 pruning): 0.105

validation error (after 2 pruning): 0.107.

test error(after 2 pruning): 0.103


The payment_delay_month feature is the most salient feature because if a person submitted late payment or no payment in the past it is very likely that the person would do it again.


In [327]:
import numpy as np
from math import log
import time
import random
from copy import deepcopy

In [340]:
class Node:
    def __init__(self, data, level):
        self.level = level
        self.parent = None
        self.le_yes_child = None
        self.gt_no_child = None
        self.data = data
        self.is_leaf = None
        self.pure = None
        self.f = None
        self.t = None
        
        if self.pure == None:
            if len(set(self.data[:,-1])) != 1:
                self.pure = False
                self.is_leaf = False
            else:
                self.pure = True
                self.is_leaf = True
                
                
    def set_data(self, data):
        self.data = data
        
        if len(set(self.data[:,-1])) != 1:
            self.pure = False
            self.is_leaf = False
        else:
            self.pure = True
            self.is_leaf = True
                
    def majority(self):
        labels = self.data[:,-1]
        ones = np.count_nonzero(labels)
        zeros = labels.shape[0] - ones
        if ones > zeros:
            return 1.0
        elif ones < zeros:
            return 0.0
        else:
            return random.choice([0.0, 1.0])
        
    def is_pure(self):
        return self.pure
        
    def set_le_yes_child(self, child):
        self.le_yes_child = child
        
    def set_gt_no_child(self, child):
        self.gt_no_child = child
        
    def print_node(self, outfile, level=10000, print_data = False):
        if self.level < level:
            if self.is_leaf:
                outfile.write('Node level: {}, is_leaf: {}, label: {}.\n'.format(self.level+1, self.is_leaf, self.majority()))
            else:
                outfile.write('Node level: {}, is_leaf: {}, rule: feature_{} <= {}.\n'.format(self.level+1, self.is_leaf, self.f+1, self.t))
            if print_data:
                outfile.write('data in node has labels: {}\n'.format(self.data[:,-1]))
            if self.le_yes_child is not None:
                self.le_yes_child.print_node(outfile, level, print_data)
            if self.le_yes_child is not None:
                self.gt_no_child.print_node(outfile, level, print_data)
            
        
    

In [341]:
def read_file(filename):
    f = open(filename, 'r')
    ls = []
    lines = f.readlines()
    for line in lines:
        if line:
            content = line.strip().split()
            ls.append([float(x) for x in content])
        
    f.close()
    ls = np.asarray(ls)
    return ls


def find_splitting_rule(node):
    num_of_features = node.data.shape[1] - 1
    max_IG = 0.0
    max_t = None
    max_f = None
    for i in range(num_of_features):
        feature_vals = list(set(node.data[:,i]))
        feature_vals_sorted = sorted(feature_vals)
        thresholds = [(feature_vals_sorted[j] + feature_vals_sorted[j+1]) / 2 for j in range(len(feature_vals_sorted)-1)]
        X_entropy = entropy(node.data)
        for t in thresholds:
            X_Z_entropy = conditional_entropy(node.data, i, t)
            IG = X_entropy - X_Z_entropy
            if IG > max_IG:
                max_IG = IG
                max_f = i
                max_t = t
    node.is_leaf = False
    node.f = max_f
    node.t = max_t
    yes_data, no_data = split_data(node.data, node.f, node.t)

    
    l_node = Node(yes_data, node.level+1)
    r_node = Node(no_data, node.level+1)
    l_node.parent = node
    r_node.parent = node
    
    node.set_le_yes_child(l_node)
    node.set_gt_no_child(r_node)
    
def split_data(data, f, t):
    features = data[:,f]
    yes_index = np.where(features <= t)
    no_index = np.where(features > t)
    yes_branch = data[yes_index]
    no_branch = data[no_index]
    
    return yes_branch, no_branch
    
            
        
        
def entropy(data, labels = None):
    if labels is None:
        labels = data[:,-1]
    total = labels.shape[0] + 0.0
    ones = np.count_nonzero(labels) + 0.0
    if ones == 0.0:
        return 0.0
    zeros = total - ones
    if zeros == 0.0:
        return 0.0
    p_zero = zeros / total
    p_one = ones / total
    ent = -p_zero * log(p_zero) - p_one * log(p_one)

    return ent

def conditional_entropy(data, i, t):
    labels = data[:,-1]
    features = data[:,i]
    
    yes_branch = np.where(features <= t)
    no_branch = np.where(features > t)
    
    yes_labels = labels[yes_branch]
    no_labels = labels[no_branch]
    
    num_yes = yes_labels.shape[0]
    num_no = no_labels.shape[0]

    p_yes = float(num_yes) / float(num_yes + num_no)
    p_no = float(num_no) / float(num_yes + num_no)

    X_yes_entropy = entropy(data, yes_labels)
    X_no_entropy = entropy(data, no_labels)
    
    rs = p_yes * X_yes_entropy + p_no * X_no_entropy

    return rs
    
def predict(root, sample):
    if root.is_pure():
        return root.data[:,-1][0]
    cur = root
    while not cur.is_leaf:
        f = cur.f
        t = cur.t
        if sample[f] <= t:
            cur = cur.le_yes_child
        else:
            cur = cur.gt_no_child
    return cur.data[:,-1][0]

def evaluate(root, eval_data):
    err_count = 0.0
    total_count = eval_data.shape[0]
    for each in eval_data:
        prediction = predict(root, each)
        if prediction != each[-1]:
            err_count += 1.0
    error = err_count / total_count
    return error
    
def prune(root, val_data):
    queue = []
    queue.append(root)
    min_error = evaluate(root, val_data)
    while queue:
        cur = queue.pop()
        majority = cur.majority()
        old_data = deepcopy(cur.data)
        cur.set_data(np.array([[majority]]))
        val_error = evaluate(root, val_data)
        if val_error < min_error:
            cur.le_yes_child = None
            cur.gt_no_child = None
            return val_error
        else:
            cur.set_data(old_data)
            if not cur.le_yes_child is None:
                queue.append(cur.le_yes_child)
            if not cur.gt_no_child is None:
                queue.append(cur.gt_no_child)
        
    

In [342]:
train_data = read_file('./data/pa2train.txt')
val_data = read_file('./data/pa2validation.txt')
test_data = read_file('./data/pa2test.txt')
num_of_samples = train_data.shape[0]
num_of_features = train_data.shape[1] - 1

In [343]:
start = time.time()
impure_nodes = []
root = Node(train_data, 0)
if root.is_pure():
    print('root is pure. No need to continue.')
else:
    impure_nodes.append(root)
    
while(impure_nodes):
    cur = impure_nodes.pop()
    if cur.is_pure():
        continue
    else:
        find_splitting_rule(cur)
        impure_nodes.append(cur.le_yes_child)
        impure_nodes.append(cur.gt_no_child)
        count += 2
    
end = time.time()
total = end - start
print('time used: {} seconds.'.format(total))

time used: 2.7438628673553467 seconds.


In [344]:
######################################
# ***************NOTE*************** #
#                                    #
# Please run this only one time or   #
# you will get wrong results. If you #
# need to run it again please run all#
# above blocks again before running  #
# this block.                        #
######################################

train_error = evaluate(root, train_data)
print('training error(before pruning): {}'.format(train_error))
test_error = evaluate(root, test_data)
print('test error(before pruning): {}'.format(test_error))
val_error = evaluate(root, val_data)
print('validation error (before pruning): {}.'.format(val_error))
print()
# pruning
# round 1
val_error = prune(root, val_data)
train_error = evaluate(root, train_data)
print('training error(after 1 pruning): {}'.format(train_error))
val_error = evaluate(root, val_data)
print('validation error (after 1 pruning): {}.'.format(val_error))
test_error = evaluate(root, test_data)
print('test error(after 1 pruning): {}'.format(test_error))
print()
# round 2
val_error = prune(root, val_data)
train_error = evaluate(root, train_data)
print('training error(after 2 pruning): {}'.format(train_error))
val_error = evaluate(root, val_data)
print('validation error (after 2 pruning): {}.'.format(val_error))
test_error = evaluate(root, test_data)
print('test error(after 2 pruning): {}'.format(test_error))



training error(before pruning): 0.0
test error(before pruning): 0.171
validation error (before pruning): 0.178.

training error(after 1 pruning): 0.0205
validation error (after 1 pruning): 0.163.
test error(after 1 pruning): 0.157

training error(after 2 pruning): 0.105
validation error (after 2 pruning): 0.107.
test error(after 2 pruning): 0.103


In [345]:
# if you want to check out the nodes in the tree, run this block
out = open('out', 'w')
# Currently it prints out the first 3 levels of nodes. 
# If you want to print out the whole tree, use:
# root.print_node(out)
root.print_node(out, level = 3)
out.close()